In [1]:
import pandas as pd
import numpy as np

FILE = 'Historical Product Demand.csv'

df_raw = pd.read_csv(FILE)

print(df_raw.head(5))
df_raw = df_raw.dropna()

print(df_raw.isna().sum())

   Product_Code Warehouse Product_Category       Date Order_Demand
0  Product_0993    Whse_J     Category_028  2012/7/27         100 
1  Product_0979    Whse_J     Category_028  2012/1/19         500 
2  Product_0979    Whse_J     Category_028   2012/2/3         500 
3  Product_0979    Whse_J     Category_028   2012/2/9         500 
4  Product_0979    Whse_J     Category_028   2012/3/2         500 
Product_Code        0
Warehouse           0
Product_Category    0
Date                0
Order_Demand        0
dtype: int64


In [2]:
def splitDate(datestr: str):
    datelist = datestr.split('/')
    for n in range(len(datelist)):
        datelist[n] = int(datelist[n])
    return datelist

df_raw['Date'] = df_raw['Date'].apply(splitDate)
df_raw['Year'] = df_raw['Date'].apply(lambda x: x[0])
df_raw['Month'] = df_raw['Date'].apply(lambda x: x[1])
df_raw['Day'] = df_raw['Date'].apply(lambda x: x[2])
df_raw = df_raw.drop('Date', axis=1)
print(df_raw.head(5))

   Product_Code Warehouse Product_Category Order_Demand  Year  Month  Day
0  Product_0993    Whse_J     Category_028         100   2012      7   27
1  Product_0979    Whse_J     Category_028         500   2012      1   19
2  Product_0979    Whse_J     Category_028         500   2012      2    3
3  Product_0979    Whse_J     Category_028         500   2012      2    9
4  Product_0979    Whse_J     Category_028         500   2012      3    2


In [3]:
print(df_raw['Product_Code'].value_counts())
print(df_raw['Product_Category'].value_counts())
def splitCode(code: str):
    codelist = code.split('_')
    return int(codelist[1])

df_raw['Product_Code'] = df_raw['Product_Code'].apply(splitCode)
df_raw['Product_Category'] = df_raw['Product_Category'].apply(splitCode)



Product_Code
Product_1359    16936
Product_1295    10575
Product_1378     9770
Product_0620     9428
Product_1286     8888
                ...  
Product_1698        2
Product_0465        1
Product_2099        1
Product_1703        1
Product_0853        1
Name: count, Length: 2160, dtype: int64
Product_Category
Category_019    470266
Category_005    101627
Category_001     97787
Category_007     82402
Category_021     52008
Category_006     35552
Category_028     31012
Category_011     23208
Category_015     22954
Category_024     20885
Category_009     19738
Category_026     14771
Category_030     12997
Category_032      9296
Category_022      8657
Category_023      7899
Category_018      5239
Category_003      4189
Category_013      3743
Category_020      3490
Category_031      2268
Category_033      1849
Category_008      1560
Category_012      1147
Category_010       976
Category_029       671
Category_017       615
Category_004       329
Category_002        77
Category_016        3

In [4]:
def replaceParens(s: str):
    s = s.replace('(','-')
    s = s.replace(')', '')
    return int(s)

df_raw['Order_Demand'] = df_raw['Order_Demand'].apply(replaceParens)
print(df_raw.head(5))

   Product_Code Warehouse  Product_Category  Order_Demand  Year  Month  Day
0           993    Whse_J                28           100  2012      7   27
1           979    Whse_J                28           500  2012      1   19
2           979    Whse_J                28           500  2012      2    3
3           979    Whse_J                28           500  2012      2    9
4           979    Whse_J                28           500  2012      3    2


In [5]:
def encodeOneHot(df, col: str, prefix: str = None):
    df = pd.concat([df, pd.get_dummies(df[col], dtype=float, prefix=prefix)], axis=1)
    df = df.drop(col, axis=1)
    return df

onehot_features = ['Warehouse']
for f in onehot_features:
    df_raw = encodeOneHot(df_raw, f, prefix=f)

corrs = df_raw.corrwith(df_raw['Order_Demand']).sort_values()
print(corrs)

Warehouse_Whse_A   -0.053673
Warehouse_Whse_J   -0.031373
Day                -0.002012
Month              -0.000690
Year                0.008365
Product_Code        0.031889
Product_Category    0.053927
Warehouse_Whse_C    0.062563
Warehouse_Whse_S    0.071347
Order_Demand        1.000000
dtype: float64


In [6]:
from sklearn.preprocessing import MinMaxScaler

scaler = MinMaxScaler()
df_scaled = pd.DataFrame(scaler.fit_transform(df_raw.to_numpy()), columns=df_raw.columns.to_list())

In [7]:
from sklearn.model_selection import train_test_split
df_train, df_valid_test = train_test_split(df_scaled, test_size=0.2, random_state=0)
df_valid, df_test = train_test_split(df_valid_test, test_size=0.5, random_state=0)
target = 'Order_Demand'
y_train, y_valid, y_test = df_train[target], df_valid[target], df_test[target]
X_train, X_valid, X_test = df_train.drop(target, axis=1), df_valid.drop(target, axis=1), df_test.drop(target, axis=1)


In [8]:
from sklearn.tree import DecisionTreeRegressor

best_score = 0
best_depth = 0
for d in range(8,16):
    dtr = DecisionTreeRegressor(max_depth=d).fit(X_train, y_train)
    score = dtr.score(X_valid, y_valid)
    if score > best_score:
        best_score, best_depth = score, d

print("VALIDATION: the best score was %f with max_depth=%d"%(best_score, best_depth))

dtr = DecisionTreeRegressor(max_depth=best_depth).fit(X_train, y_train)
test_score = dtr.score(X_test, y_test)
print("TESTING: the test score was %f with max_depth=%d"%(test_score, best_depth))


VALIDATION: the best score was 0.183133 with max_depth=10
TESTING: the test score was 0.161211 with max_depth=10


In [9]:
'''I tried encoding dates using OneHot encoding and the results were poor.
Dropping Day and using only Month and Year resulted in a noticeable improvement,
but the best R2 score results came from dropping date values completely.
Interestingly, the OneHot Encoded values worked best when they were also scaled,
but the results still weren't as good as dropping date values completely.
'''
features_to_drop = ['Day', 'Year', 'Month']
df_fs = df_scaled.drop(features_to_drop, axis=1)

from sklearn.model_selection import train_test_split
df_train, df_valid_test = train_test_split(df_fs, test_size=0.2, random_state=0)
df_valid, df_test = train_test_split(df_valid_test, test_size=0.5, random_state=0)
y_train, y_valid, y_test = df_train[target], df_valid[target], df_test[target]
X_train, X_valid, X_test = df_train.drop(target, axis=1), df_valid.drop(target, axis=1), df_test.drop(target, axis=1)

best_score = 0
best_depth = 0
for d in range(16,26):
    dtr = DecisionTreeRegressor(max_depth=d).fit(X_train, y_train)
    score = dtr.score(X_valid, y_valid)
    if score > best_score:
        best_score, best_depth = score, d

print("VALIDATION: the best score was %f with max_depth=%d"%(best_score, best_depth))

dtr = DecisionTreeRegressor(max_depth=best_depth).fit(X_train, y_train)
test_score = dtr.score(X_test, y_test)
print("TESTING: the test score was %f with max_depth=%d"%(test_score, best_depth))


VALIDATION: the best score was 0.205758 with max_depth=24
TESTING: the test score was 0.226794 with max_depth=24


In [ ]:
from sklearn.svm import SVR

svr = SVR().fit(X_train, y_train)
test_score = svr.score(X_test, y_test)
print("TESTING: the test score was %f"%test_score)
'''I only fit radial-basis function SVR here because the linear and
polynomial options were taking over 33 minutes to run so I left those
out as impractical. I'll try a random-forest approach below.
The RBF SVR scored terribly, however.
'''

TESTING: the test score was -202.176278


In [13]:
from sklearn.ensemble import RandomForestRegressor

best_score = 0
best_estimators = 0
best_depth = 0
best_rfr = None

for n in range(3,7):
    for d in range(20,26):
        rfr = RandomForestRegressor(n_estimators=n, max_depth=d).fit(X_train, y_train)
        score = rfr.score(X_valid, y_valid)
        if score > best_score:
            best_score, best_estimators, best_depth, best_rfr = score, n, d, rfr

print("VALIDATION: random forest best score %f with n_estimators=%d and max_depth=%d"%(best_score, best_estimators, best_depth))

test_score = rfr.score(X_test, y_test)
print("Testing: random forest test score %f with n_estimators=%d and max_depth=%d"%(test_score, best_estimators, best_depth))

VALIDATION: random forest best score 0.208175 with n_estimators=5 and max_depth=22
Testing: random forest test score 0.226280 with n_estimators=5 and max_depth=22
